Clase 11
======

Objetivos

* Alineamiento global vs alineamiento local
* Algortimo de alineamiento local de Smith-Waterman

Anuncios
----------

1. Felicitaciones a quienes ya han entregado una versión funcional de la tarea optativa de la clase 10. Para no la han entregado, al final de ésta clase se ofrecerá otra tarea optativa, la cual pueden entregar hasta el próximo jueves hasta las 11pm. 
2. La primera evaluación ser hará el próximo lunes 18 de mayo en el horario 16:20 - 17:50pm.
3. El jueves a la las 20:00 habrá un horario de consultas por zoom (el mismo link de la clase).


Resumen clase anterior
=============

Algoritmo de Needleman-Wunsch
------

Supongamos que queremos encontrar el mejor alineamineto entre $x="HEAGAWGHEE"$ e $y="PAWHEAE"$, utilizando la matriz BLOSUM50 (scores para match/mismatch) y $d=-8$ (score para gaps). En este caso podemos escribir todas la alineaciones posibles y los scores de los mejores alineamientos parciales como la matriz:

<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_exampleAlignment.jpg" alt="Drawing" style="width: 500px;" />

En donde la coordenada $(i,j)$ corresponde al mejor score entre las secuencias $x_{1...i}$ y $y_{1...j}$. Calculamos el valor de cada coordenadas en base a sus valores adyacentes:

<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_trace.jpg" alt="Drawing" style="width: 400px"/>

De estas tres opciones, se selecciona el score que hasta ese momento sea el más alto:

<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_selection.jpg" alt="Drawing" style="width: 400px;"/>

Veamos el ejemplo paso a paso a través de los siguientes ejercicios.


Tarea optativa
----------

Dada las secuencias $x="HEAGAWGHEE"$ e $y="PAWHEAE"$, escribe código para:

1. Construir la matriz con los scores de cada secuencia (Matriz BLOSUM50).
2. Computar la matriz de los los mejores alineamientos parciales (Matriz F).
3. Trazar la suceción de alineamientos parciales con el mayor score total.
4. Escribe una función ```align(seq1,seq2)``` que tome como argumentos las secuencias ```seq1``` y ```seq2```, y entrgue como resultado su mejor alineamiento. 

In [74]:
# 1
import numpy as np
import pandas as pd
from Bio.SubsMat import MatrixInfo

seq1="HEAGAWGHEE"
seq2="PAWHEAE"

def createList(seq): # toma como argumento un string
    x=[]
    for i in seq:
        x.append(i)
    return(x) # devuelve una lista

seq1=createList(seq1)
seq2=createList(seq2)

scoreMatrix=np.zeros((len(seq2),len(seq1) ) )  

for i in range(len(seq2)):
    for j in range(len(seq1)):
        pair = (seq2[i],seq1[j])
        if not pair in MatrixInfo.blosum50:
            pair = tuple(reversed(pair))
        scoreMatrix[i,j]=MatrixInfo.blosum50[pair]

scoreMatrix=pd.DataFrame(scoreMatrix,index=seq2,columns=seq1) 
scoreMatrix

,H,E,A,G,A,W,G,H,E,E
P,-2.0,-1.0,-1.0,-2.0,-1.0,-4.0,-2.0,-2.0,-1.0,-1.0
A,-2.0,-1.0,5.0,0.0,5.0,-3.0,0.0,-2.0,-1.0,-1.0
W,-3.0,-3.0,-3.0,-3.0,-3.0,15.0,-3.0,-3.0,-3.0,-3.0
H,10.0,0.0,-2.0,-2.0,-2.0,-3.0,-2.0,10.0,0.0,0.0
E,0.0,6.0,-1.0,-3.0,-1.0,-3.0,-3.0,0.0,6.0,6.0
A,-2.0,-1.0,5.0,0.0,5.0,-3.0,0.0,-2.0,-1.0,-1.0
E,0.0,6.0,-1.0,-3.0,-1.0,-3.0,-3.0,0.0,6.0,6.0


In [75]:
# 2 y 3
seq1="-HEAGAWGHEE"
seq2="-PAWHEAE"

seq1=createList(seq1)
seq2=createList(seq2)

d=8
traceMatrix=np.zeros((len(seq2),len(seq1)) ) 
Fmatrix=np.zeros((len(seq2),len(seq1)) )  

# Set first column
Fmatrix[:,0] = -d*np.array(range( len(seq2) ))
# Set first row

Fmatrix[0,:] = -d*np.array(range( len(seq1) ))

for i in range(1,len(seq2)):
    for j in range(1,len(seq1)):
        value_i=seq2[i]
        value_j=seq1[j]
        pair = (value_i,value_j)
        if not pair in MatrixInfo.blosum50:
            pair = tuple(reversed(pair))
        score=MatrixInfo.blosum50[pair]
        Fmatrix[i,j] = np.max([  Fmatrix[i,j-1]-d,Fmatrix[i-1,j-1]+score,Fmatrix[i-1,j]-d  ])
        indexMax=       np.argmax([Fmatrix[i,j-1]-d,Fmatrix[i-1,j-1]+score,Fmatrix[i-1,j]-d])
        traceMatrix[i,j] = indexMax
traceMatrix = pd.DataFrame(traceMatrix,index=seq2,columns=seq1) 
Fmatrix = pd.DataFrame(Fmatrix,index=seq2,columns=seq1) 
Fmatrix

,-,H,E,A,G,A,W,G,H,E,E
-,0.0,-8.0,-16.0,-24.0,-32.0,-40.0,-48.0,-56.0,-64.0,-72.0,-80.0
P,-8.0,-2.0,-9.0,-17.0,-25.0,-33.0,-41.0,-49.0,-57.0,-65.0,-73.0
A,-16.0,-10.0,-3.0,-4.0,-12.0,-20.0,-28.0,-36.0,-44.0,-52.0,-60.0
W,-24.0,-18.0,-11.0,-6.0,-7.0,-15.0,-5.0,-13.0,-21.0,-29.0,-37.0
H,-32.0,-14.0,-18.0,-13.0,-8.0,-9.0,-13.0,-7.0,-3.0,-11.0,-19.0
E,-40.0,-22.0,-8.0,-16.0,-16.0,-9.0,-12.0,-15.0,-7.0,3.0,-5.0
A,-48.0,-30.0,-16.0,-3.0,-11.0,-11.0,-12.0,-12.0,-15.0,-5.0,2.0
E,-56.0,-38.0,-24.0,-11.0,-6.0,-12.0,-14.0,-15.0,-12.0,-9.0,1.0


In [64]:
traceMatrix

,-,H,E,A,G,A,W,G,H,E,E
-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
W,0.0,2.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
H,0.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0
E,0.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0
A,0.0,2.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,2.0,1.0
E,0.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [153]:
# Solución de la tarea optativa de la clase 10
x="-HEAGAWGHEE"
y="-PAWHEAE"

def globalAligner(x,y,traceMatrix):
    i = len(y)-1 # 7
    j = len(x)-1 #10
    alignedX,alignedY=[],[] # secueencias alineadas
    traceMatrix=np.matrix(traceMatrix)
    while not (i,j)==(0,0):
        if traceMatrix[i,j]==0: # movimiento lateral: GAP en Y 
            alignedX.append(x[j])
            alignedY.append("-")
            i = i
            j = j -1
        elif traceMatrix[i,j]==1: # movimiento diagonal: MATCH
            alignedX.append(x[j])
            alignedY.append(y[i])
            i = i-1
            j = j-1
        elif traceMatrix[i,j]==2: # movimiento vertical: gap en X
            alignedX.append("-")
            alignedY.append(y[i])
            i = i - 1
            j = j
    alignedX = list(reversed(alignedX)) 
    alignedY = list(reversed(alignedY)) 
    return(alignedX,alignedY)

In [99]:
x="-HEAGAWGHEE"
y="-PAWHEAE"

solX,solY=globalAligner(x,y,traceMatrix)

In [100]:
print(solX)
print(solY)

['H', 'E', 'A', 'G', 'A', 'W', 'G', 'H', 'E', '-', 'E']
['-', 'P', 'A', '-', '-', 'W', '-', 'H', 'E', 'A', 'E']


In [150]:
i=10
while i>0:
    print(i)
    i = i - 1

10
9
8
7
6
5
4
3
2
1


In [152]:
(4,3)==(5,6)

False

Local alignment: Algoritmo de Smith-Waterman
================

Hasta ahora hemos asumido que conocemos las dos secuencias que queremos alinear, $x$ e $y$, y que estamos buscando por el mejor alineamiento cada una de sus secuencias de principio a fin. Pero una situación mucho más común es cuando estamos buscando el mejor alineamiento entre una subsecuencia de $x$ e $y$. Esto ocurre, por ejemplo, cuando sospechamos que dos dominios de dos proteinas comparten un dominio común, o también ocurre cuando comparamos dos secciones de ADN. Es también la mejor manera para detectar similaridades entre secuencias que a través de la evolución se han vuelto muy divergentes. Esto puede ocurrir, por ejemplo,  en aquellos casos en los cuales distintas regiones del genome tienen distintas sensivilidaddes a mutaciones, tal como es el caso para intrones y exones. En este útlimo caso mutaciones en los exones tienden a ser menos comunes que mutaciones en los intrones, para lo cual solo estariamos interesados en hacer alineamientos entre exones.

Para todos estos casos, la mejor estrategia es alinear subsecuencias de $x$ e $y$. Este estrategia se llama alineamiento local. El algoritmo para encontrar el mejor alineamiento local es muy similar al de alineamiento global. Sólo hay dos diferencias.

**Primero**, en cada celda de la matrix F, un posibilidad extra es agregada para permitir que $F(i,j)$ pueda tomar el valor $0$ si todas las otras opciones son menores que $0$:

$
F(i,j)=max
\begin{cases}
  0,\\
  F(i-1,j-1)+s(x_i,y_i) \\
  F(i-1,j)-d \\
  F(i,j-1)-d
\end{cases}
$

Tomar la opción $0$ corresponde a comenzar un nuevo alineamiento. Si el mejor alineamiento hasta algún punto tiene un score negativo, es mejor comenzar un nuevo alineamiento en lugar de extender el actual. Nota que como consecuencia de esta opción adicional la primera fila y columna de la matriz $F$ ahora estarán llenas de ceros, no por $-d*i$ o $-d*j$ como era el caso para el alineamiento global. 

**El segundo cambio** es que ahora el mejor alineamiento puede estar en cualquier parte de la matriz $F$. Así, en lugar de tomar el valor en el rincón derecho inferior de la matriz $F$ ($F(n,m)$), ahora se debe considerar como punto de partida el valor más alto de $F(i,j)$ a lo largo de toda la matriz. El trazado termina cuando encontramos un valor $0$, lo cual corresponde al comienzo del alineamiento.

Por ejemplo, veamos como se ve la matriz $F$ y alineamiento de las secuencias $x=HEAGAWGHEE$ e $y=PAWHEAE$.

<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase10_ejemplo.jpg" alt="Drawing" style="width: 400px;"/>

Ejercicio
---------

1. Escribe una función $localF(x,y)$ para calcular la matriz $F$ de  las secuencias $x=HEAGAWGHEE$ e $y=PAWHEAE$. Considera $d=-8$ y la matriz BLOSUM50. 


Tarea opcional
-----------
2. Utiliza el resultado anterior para crear la función $localAlignment(x,y)$ para entregar el alineamiento de las secuencias $x=HEAGAWGHEE$ e $y=PAWHEAE$.

In [108]:
# Ejercicio 1

# Tarea opcional

# 2 y 3
seq1="-HEAGAWGHEE"
seq2="-PAWHEAE"

seq1=createList(seq1)
seq2=createList(seq2)

d=8
traceMatrix=np.zeros((len(seq2),len(seq1)) ) 
Fmatrix=np.zeros((len(seq2),len(seq1)) )  

for i in range(1,len(seq2)):
    for j in range(1,len(seq1)):
        value_i=seq2[i]
        value_j=seq1[j]
        pair = (value_i,value_j)
        if not pair in MatrixInfo.blosum50:
            pair = tuple(reversed(pair))
        score=MatrixInfo.blosum50[pair]
        Fmatrix[i,j] = np.max([0,  Fmatrix[i,j-1]-d,Fmatrix[i-1,j-1]+score,Fmatrix[i-1,j]-d  ])
        indexMax=       np.argmax([0,Fmatrix[i,j-1]-d,Fmatrix[i-1,j-1]+score,Fmatrix[i-1,j]-d])
        traceMatrix[i,j] = indexMax
traceMatrix = pd.DataFrame(traceMatrix,index=seq2,columns=seq1) 
np.matrix(traceMatrix)
Fmatrix = pd.DataFrame(Fmatrix,index=seq2,columns=seq1) 
Fmatrix


,-,H,E,A,G,A,W,G,H,E,E
-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A,0.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
W,0.0,0.0,0.0,0.0,2.0,0.0,20.0,12.0,4.0,0.0,0.0
H,0.0,10.0,2.0,0.0,0.0,0.0,12.0,18.0,22.0,14.0,6.0
E,0.0,2.0,16.0,8.0,0.0,0.0,4.0,10.0,18.0,28.0,20.0
A,0.0,0.0,8.0,21.0,13.0,5.0,0.0,4.0,10.0,20.0,27.0
E,0.0,0.0,6.0,13.0,18.0,12.0,4.0,0.0,4.0,16.0,26.0


In [ ]:
# Tarea opcional

def localAlignment(x,y):
    
    
    
    
    return(aligendX,alignedY)

In [ ]:
x=HEAGAWGHEE
y=PAWHEAE
solx,soly=localAlignment(x,y)

In [145]:
def score_aa(seq5,seq6): 
    Puntaje = 0 
    for i in range(len(seq5)): 
        #for j in range(len(seq6)): 
        if seq5[i] == "-": 
            Puntaje = Puntaje - 8 
        elif seq6[i] == "-":
            Puntaje = Puntaje - 8 
        else: 
            pair = (seq5[i],seq6[i])
            if not pair in  MatrixInfo.blosum50:
                pair = tuple(reversed(pair) ) # da vuelta el orden del pair
            score = MatrixInfo.blosum50[pair] 
            print(pair,score)
            Puntaje = Puntaje + score

    return(Puntaje) 

In [146]:

MatrixInfo.blosum50[ tuple(reversed( ('F', 'W') ))]

1

In [147]:
seq5 = "VFRKAGND"
seq6 = "VKRKAAND"
score_aa(seq5,seq6)

('V', 'V') 5
('F', 'K') -4
('R', 'R') 7
('K', 'K') 6
('A', 'A') 5
('G', 'A') 0
('N', 'N') 7
('D', 'D') 8


34

In [119]:
MatrixInfo.blosum50[('G', 'A')]

0

In [118]:
MatrixInfo.blosum50

{('W', 'F'): 1,
 ('L', 'R'): -3,
 ('S', 'P'): -1,
 ('V', 'T'): 0,
 ('Q', 'Q'): 7,
 ('N', 'A'): -1,
 ('Z', 'Y'): -2,
 ('W', 'R'): -3,
 ('Q', 'A'): -1,
 ('S', 'D'): 0,
 ('H', 'H'): 10,
 ('S', 'H'): -1,
 ('H', 'D'): -1,
 ('L', 'N'): -4,
 ('W', 'A'): -3,
 ('Y', 'M'): 0,
 ('G', 'R'): -3,
 ('Y', 'I'): -1,
 ('Y', 'E'): -2,
 ('B', 'Y'): -3,
 ('Y', 'A'): -2,
 ('V', 'D'): -4,
 ('B', 'S'): 0,
 ('Y', 'Y'): 8,
 ('G', 'N'): 0,
 ('E', 'C'): -3,
 ('Y', 'Q'): -1,
 ('Z', 'Z'): 5,
 ('V', 'A'): 0,
 ('C', 'C'): 13,
 ('M', 'R'): -2,
 ('V', 'E'): -3,
 ('T', 'N'): 0,
 ('P', 'P'): 10,
 ('V', 'I'): 4,
 ('V', 'S'): -2,
 ('Z', 'P'): -1,
 ('V', 'M'): 1,
 ('T', 'F'): -2,
 ('V', 'Q'): -3,
 ('K', 'K'): 6,
 ('P', 'D'): -1,
 ('I', 'H'): -4,
 ('I', 'D'): -4,
 ('T', 'R'): -1,
 ('P', 'L'): -4,
 ('K', 'G'): -2,
 ('M', 'N'): -2,
 ('P', 'H'): -2,
 ('F', 'Q'): -4,
 ('Z', 'G'): -2,
 ('X', 'L'): -1,
 ('T', 'M'): -1,
 ('Z', 'C'): -3,
 ('X', 'H'): -1,
 ('D', 'R'): -2,
 ('B', 'W'): -5,
 ('X', 'D'): -1,
 ('Z', 'K'): 1,
 ('F', 'A'):